In [3]:
import torch as t
print(torch.__version__)
print(torch.cuda.is_available())  # should be False on macOS
print(torch.backends.mps.is_available())  # True if Apple Silicon

2.8.0
False
True


In [2]:
def to_onehot(y, num_classes):
    y_onehot = torch.zeros(y.size(0), num_classes)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1).float()
    return y_onehot

y = torch.tensor([0, 1, 2, 2])

y_enc = to_onehot(y, 3)

print('one-hot encoding:\n', y_enc)

one-hot encoding:
 tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]])


In [4]:
t0 = t.tensor(1)

In [11]:
t1 = t.tensor([1,2,3])
t2 = t.tensor([[1,2],[3,4]])
t3 = t.tensor([[[1,2],[3,4]],
               [[5,6], [7,8]]])

In [12]:
print(t1.dtype)

torch.int64


In [13]:
floatvec = t1.to(t.float32)

In [14]:
print(floatvec)

tensor([1., 2., 3.])


In [18]:
t22 = t.tensor([[1,2,3], [4,5,6]])

In [19]:
print(t22.reshape(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [20]:
print(t22.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [21]:
print(t2.matmul(t2.T))

tensor([[ 5, 11],
        [11, 25]])
